# 環境構築
## 目的

`C:\lib_ana\lib.ipynb` から、指定した Pythonライブラリの公開API（外部から呼ぶ想定の関数・クラス等）を抽出し、一覧表（CSV/Markdown/Excel など）として `outputs/` に出力する。

## 設計方針（再現性・保守性）

* Notebook は「操作UI」、実装は `src/` に寄せる（ロジック分離）
* 生成物はすべて `outputs/` に集約（差分管理・掃除が簡単）
* 実行ログは `logs/`（失敗時の原因追跡）
* 一時物は `tmp/`（キャッシュや中間ファイル）

## 提案ツリー

```text
C:\lib_ana\
  lib.ipynb
  .vscode\
    settings.json

  src\
    lib_ana\
      __init__.py
      paths.py          # パス定義
      logging_conf.py   # logging(ログ記録)設定
      api_scan.py       # API抽出(探索/inspect等)
      render.py         # 表の整形(CSV/MD/Excel)

  configs\
    settings.json       # 解析対象・出力形式など（任意）

  outputs\
    api_tables\         # API一覧(表)
    reports\            # レポート(まとめ)
    artifacts\          # 追加生成物(図/HTML等)

  data\
    sample\             # サンプル入力(任意)

  logs\
  tmp\
```


In [4]:
# =========================================
# Project bootstrap (Windows / Jupyter)
# 目的: 解析用のディレクトリ構造を自動生成し、
#      Notebookから src/ をimportできるようにする
# =========================================

from __future__ import annotations

import os
import sys
import json
import platform
from pathlib import Path
from datetime import datetime
import logging

# --- 1) ルート固定（lib.ipynbが C:\lib_ana\lib.ipynb 前提）
PROJECT_ROOT = Path(r"C:\lib_ana").resolve()
NOTEBOOK_PATH = PROJECT_ROOT / "lib.ipynb"

# --- 2) ディレクトリ設計（必要なら増やす）
DIRS = {
    "src": PROJECT_ROOT / "src",
    "pkg": PROJECT_ROOT / "src" / "lib_ana",
    "configs": PROJECT_ROOT / "configs",
    "outputs": PROJECT_ROOT / "outputs",
    "api_tables": PROJECT_ROOT / "outputs" / "api_tables",
    "reports": PROJECT_ROOT / "outputs" / "reports",
    "artifacts": PROJECT_ROOT / "outputs" / "artifacts",
    "data": PROJECT_ROOT / "data",
    "sample": PROJECT_ROOT / "data" / "sample",
    "logs": PROJECT_ROOT / "logs",
    "tmp": PROJECT_ROOT / "tmp",
    "vscode": PROJECT_ROOT / ".vscode",
}

for p in DIRS.values():
    p.mkdir(parents=True, exist_ok=True)

# --- 3) Pythonパッケージ雛形を作る（import可能にする）
init_file = DIRS["pkg"] / "__init__.py"
if not init_file.exists():
    init_file.write_text("# lib_ana package\n", encoding="utf-8")

# --- 4) src を import探索パスへ追加（sys.path）
SRC_PATH = str(DIRS["src"])
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

# --- 5) logging(ログ記録) をセットアップ
log_file = DIRS["logs"] / f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file, encoding="utf-8"),
        logging.StreamHandler(sys.stdout),
    ],
)
logger = logging.getLogger("bootstrap")
logger.info("PROJECT_ROOT=%s", PROJECT_ROOT)
logger.info("Python=%s", sys.version.replace("\n", " "))
logger.info("Platform=%s", platform.platform())
logger.info("Log file=%s", log_file)

# --- 6) ディレクトリ構造を表示
def print_tree(root: Path, max_depth: int = 4) -> None:
    root = root.resolve()
    print(f"\n[tree] {root}\n")
    for path in sorted(root.rglob("*")):
        rel = path.relative_to(root)
        depth = len(rel.parts)
        if depth > max_depth:
            continue
        indent = "  " * (depth - 1)
        marker = "📁" if path.is_dir() else "📄"
        print(f"{indent}{marker} {rel}")

print_tree(PROJECT_ROOT, max_depth=4)


2026-02-04 13:59:13,745 | INFO | bootstrap | PROJECT_ROOT=C:\lib_ana
2026-02-04 13:59:13,747 | INFO | bootstrap | Python=3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)]
2026-02-04 13:59:13,748 | INFO | bootstrap | Platform=Windows-10-10.0.26200-SP0
2026-02-04 13:59:13,749 | INFO | bootstrap | Log file=C:\lib_ana\logs\run_20260204_135913.log

[tree] C:\lib_ana

📁 .vscode
  📄 .vscode\settings.json
  📄 .vscode\settings.json.bak_20260204_135731
📁 configs
📁 data
  📁 data\sample
📄 lib.ipynb
📁 logs
  📄 logs\run_20260204_135711.log
  📄 logs\run_20260204_135913.log
📁 outputs
  📁 outputs\api_tables
  📁 outputs\artifacts
  📁 outputs\reports
📁 src
  📁 src\lib_ana
    📄 src\lib_ana\__init__.py
📁 tmp


## 目的

* `src/` を Python の解析パスに入れて補完・型解析を効かせる
* Notebook の作業ディレクトリをワークスペースに固定して、相対パス事故を防ぐ
* 保存時の自動整形（環境に整形ツールが入っていれば効く）

## 反映先

* `C:\lib_ana\.vscode\settings.json`



In [5]:
from pathlib import Path
import json
from datetime import datetime

PROJECT_ROOT = Path(r"C:\lib_ana").resolve()
vscode_dir = PROJECT_ROOT / ".vscode"
vscode_dir.mkdir(parents=True, exist_ok=True)
settings_path = vscode_dir / "settings.json"

# 既存があればバックアップ
if settings_path.exists():
    backup = settings_path.with_suffix(f".json.bak_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
    backup.write_text(settings_path.read_text(encoding="utf-8"), encoding="utf-8")

# 推奨設定（環境により拡張機能が無くても害は少ない設定に寄せる）
settings = {
    "python.analysis.extraPaths": ["./src"],
    "jupyter.notebookFileRoot": "${workspaceFolder}",
    "files.eol": "\n",
    "editor.formatOnSave": True,
    # black(コード整形ツール)等がある場合に効く。無くても致命傷にはならない。
    "python.formatting.provider": "black",
}

settings_path.write_text(json.dumps(settings, ensure_ascii=False, indent=2), encoding="utf-8")
print(f"Updated: {settings_path}")
print(settings_path.read_text(encoding='utf-8'))


Updated: C:\lib_ana\.vscode\settings.json
{
  "python.analysis.extraPaths": [
    "./src"
  ],
  "jupyter.notebookFileRoot": "${workspaceFolder}",
  "files.eol": "\n",
  "editor.formatOnSave": true,
  "python.formatting.provider": "black"
}


# ディレクトリ構造確認

In [16]:
import subprocess
import sys

# 1. shell=True: コマンドプロンプト経由で実行
# 2. capture_output=True: 出力を変数に格納
# 3. text=True: テキストとして扱う
# 4. encoding='cp932': Windowsの日本語環境(Shift-JIS)に合わせる
cp = subprocess.run(['tree', '/f'], shell=True, capture_output=True, text=True, encoding='cp932')

if cp.returncode != 0:
    print('tree failed.', file=sys.stderr)
    print(cp.stderr) # エラー内容を表示
else:
    # 成功した場合、キャプチャした標準出力を表示
    print(cp.stdout)

Folder PATH listing
Volume serial number is D4CC-DB5B
C:.
│   lib.ipynb
│   organize_files.py
│   
├───.history
│   │   organize_files_20260204162147.py
│   │   organize_files_20260204162150.py
│   │   
│   └───src
│           analyzer_v2_20260204145440.py
│           analyzer_v2_20260204145442.py
│           analyzer_v2_20260204145533.py
│           analyzer_v2_20260204145547.py
│           analyzer_v2_20260204145549.py
│           analyzer_v2_20260204145550.py
│           analyzer_v2_20260204145704.py
│           analyzer_v2_20260204150317.py
│           analyzer_v3_20260204151629.py
│           analyzer_v3_20260204151633.py
│           library_explorer_20260204144502.py
│           library_explorer_20260204144505.py
│           ui_v2_20260204145455.py
│           ui_v2_20260204145458.py
│           ui_v2_20260204145724.py
│           ui_v2_20260204150328.py
│           ui_v2_20260204150701.py
│           ui_v3_20260204151656.py
│           ui_v3_20260204151700.py
│           
├───.v

In [18]:
!Expand-Archive -Path "C:\lib_ana\src\library_explorer_v4.zip" -DestinationPath "C:\lib_ana\src\library_explorer_v4" -Force

'Expand-Archive' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import sys
import os

# srcディレクトリをパスに追加してインポートできるようにする
src_path = os.path.abspath(os.path.join(os.getcwd(), 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

# 作成したモジュールをインポclearート
# ※ファイル名を変更した場合はここも変更してください
from library_explorer import LibraryExplorerApp

# アプリケーションの起動
app = LibraryExplorerApp()
app.display()

In [ ]:
# ファイルパス: C:\lib_ana\lib.ipynb

import sys
import os

# プロジェクトルートのsrcフォルダをパスに追加
current_dir = os.getcwd()
src_path = os.path.join(current_dir, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

# 開発用リロード処理
import importlib
import ui_v2
import analyzer_v2
importlib.reload(analyzer_v2)
importlib.reload(ui_v2)

# アプリケーション起動
from ui_v2 import CognitiveLibraryUI

app = CognitiveLibraryUI()
app.display()

In [ ]:
import sys
import os

# srcパスの追加
current_dir = os.getcwd()
src_path = os.path.join(current_dir, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

# リロードと起動
import importlib
import ui_v2
importlib.reload(ui_v2)

from ui_v2 import CognitiveLibraryUI
app = CognitiveLibraryUI()
app.display()

In [6]:
from chronos import BaseChronosPipeline

# Initialize BaseChronosPipeline
instance = BaseChronosPipeline(
    inner_model=..., # Type: &#x27;PreTrainedModel&#x27;
)

In [ ]:
from ui_v3 import CognitiveLibraryUIV3
CognitiveLibraryUIV3().display()


In [5]:
# ファイルパス: C:\lib_ana\lib.ipynb

import sys
import os

# パス設定
current_dir = os.getcwd()
src_path = os.path.join(current_dir, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

# リロード（開発用）
import importlib
import ui_v3
import analyzer_v3
importlib.reload(analyzer_v3)
importlib.reload(ui_v3)

from ui_v3 import CognitiveLibraryUI_v3

# アプリ起動
app = CognitiveLibraryUI_v3()
app.display()

ImportError: cannot import name 'CognitiveLibraryUI_v3' from 'ui_v3' (c:\lib_ana\ui_v3.py)

In [11]:
import sys
sys.path.insert(0, r"C:\lib_ana\src")

from ui_v4 import CognitiveLibraryUIV4
CognitiveLibraryUIV4().display()


In [19]:
pip list


Package                            Version      Editable project location
---------------------------------- ------------ -------------------------------------------------------------------
absl-py                            2.3.1
accelerate                         1.12.0
access-deep-analyzer               4.0.0        C:\Work\AccessDeepAnalyzer
access-migration-suite             0.1.0        C:\access\src\mdb_ver\access_migration_suite
access-migration-system            0.2.0        C:\access\src\mdb_ver\access_migration_system_v3
access_parser                      0.0.6
access_parser_c                    0.0.7
access-super-eda                   0.3.0        C:\access\src\mdb
AccessAnalyzer                     1.0.0        C:\access\src\mdb_v3\AccessAnalyzer
adagio                             0.2.6
aiofiles                           24.1.0
aiohappyeyeballs                   2.6.1
aiohttp                            3.13.3
aiohttp-retry                      2.9.1
aiosignal              

In [ ]:
import sys
import os
import pandas as pd
import importlib.metadata
from tqdm.notebook import tqdm
import warnings

# srcパスの追加（v4を使用）
current_dir = os.getcwd()
src_path = os.path.join(current_dir, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

# v4のアナライザーをインポート
try:
    from analyzer_v4 import DeepLibraryAnalyzerV4
except ImportError:
    # 整理スクリプト実行後のパスに対応
    try:
        from v4.analyzer_v4 import DeepLibraryAnalyzerV4
    except ImportError:
        print("analyzer_v4が見つかりません。パスを確認してください。")

# 警告を抑制（ライブラリごとの非推奨警告などが大量に出るため）
warnings.filterwarnings("ignore")

def analyze_all_libraries():
    # 1. インストール済みライブラリのリスト取得
    print("📦 インストール済みライブラリを収集中...")
    installed_packages = []
    try:
        dists = importlib.metadata.distributions()
        for dist in dists:
            installed_packages.append(dist.metadata['Name'])
    except Exception as e:
        print(f"パッケージ一覧の取得に失敗: {e}")
        return

    unique_packages = sorted(list(set(installed_packages)))
    print(f"✨ 合計 {len(unique_packages)} 個のパッケージが見つかりました。")

    # 2. 解析実行
    all_data = []
    error_logs = []

    print("🚀 解析を開始します（時間がかかります）...")
    
    # tqdmで進捗バーを表示
    for lib_name in tqdm(unique_packages):
        try:
            # アナライザーの初期化と解析
            analyzer = DeepLibraryAnalyzerV4(lib_name)
            
            # importに失敗した場合はスキップ
            if analyzer.target_lib is None:
                continue

            # 構造解析 (深さは浅めに2に設定して高速化、必要なら増やしてください)
            # v4の scan_structure メソッドを使用
            df = analyzer.scan_structure(max_depth=2)
            
            if not df.empty:
                # どのライブラリのデータか分かるようにカラム追加
                df.insert(0, 'Library', lib_name)
                
                # 必要なカラムのみ抽出（データ量削減）
                # v4のカラム構成: Name, Type, Path, Category, ArgCount, Complexity, DocSummary, etc.
                cols = ['Library', 'Name', 'Type', 'Category', 'Path', 'ArgCount', 'Complexity', 'DocSummary']
                # 存在しないカラムは除外
                available_cols = [c for c in cols if c in df.columns]
                
                all_data.append(df[available_cols])
                
        except Exception as e:
            error_logs.append(f"{lib_name}: {str(e)}")
            continue

    # 3. データ統合と保存
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        
        # 結果の表示
        print(f"\n✅ 解析完了！")
        print(f"📊 取得した総アイテム数: {len(final_df)}")
        print(f"❌ エラー/インポート不可: {len(unique_packages) - len(set(final_df['Library']))} 個")
        
        # CSV保存
        output_dir = os.path.join(current_dir, 'outputs')
        os.makedirs(output_dir, exist_ok=True)
        csv_path = os.path.join(output_dir, 'all_libraries_analysis.csv')
        final_df.to_csv(csv_path, index=False)
        print(f"💾 CSVに保存しました: {csv_path}")
        
        return final_df
    else:
        print("❌ 解析データが得られませんでした。")
        return pd.DataFrame()

# --- 実行 ---
df_all = analyze_all_libraries()

# 上位データの確認表示
if not df_all.empty:
    import ipywidgets as widgets
    from IPython.display import display
    
    print("\n🔍 データのプレビュー (最初の10件):")
    display(df_all.head(10))
    
    # 統計情報の表示
    print("\n📈 ライブラリ別のアイテム数 (Top 10):")
    display(df_all['Library'].value_counts().head(10))
    
    print("\n💡 複雑度が高い関数 (Top 10):")
    if 'Complexity' in df_all.columns:
        display(df_all[df_all['Type'].isin(['function', 'method'])].sort_values('Complexity', ascending=False)[['Library', 'Name', 'Complexity']].head(10))